In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=(10,4)

In [ ]:
df=pd.read_csv('../input/covid19-data-keraa/covid3.csv')
df.head()
#df.columns

In [ ]:
df2=df[['Date','Confirmed']]
df2.head()

In [ ]:
#making date as index
df2['Date']=pd.to_datetime(df['Date'],infer_datetime_format=True)
ind=df2.set_index(['Date'])
ind.tail()

In [ ]:
#coverting into daily covid numbers
ind2=ind.diff(axis = 0, periods = 1)
ind2.tail()

In [ ]:
dfn=ind2.tail(50)
dfn

In [ ]:
ind2.dropna(inplace=True)
ind2.head()

In [ ]:
plt.xlabel("Date")
plt.ylabel("Confirmed")
plt.title("DailyCases")
plt.plot(ind2)

In [ ]:
# determining rolling mean statistics
rolmean=ind2.rolling(window=12).mean()
rolstd=ind2.rolling(window=12).std()
print(rolmean,rolstd)

In [ ]:
orig=plt.plot(ind2,color='blue',label='Original')
mean=plt.plot(rolmean,color='red',label='Rolling mean')
std=plt.plot(rolstd,color='green',label='Roling std')
plt.legend(loc='best')
plt.title("rolling mean and std")
plt.show(block=False)

In [ ]:
# To check stationarity

def test_stationarity(tseries):
    # determining rolling mean statistics
    rolmean=tseries.rolling(window=12).mean()
    rolstd=tseries.rolling(window=12).std()
    
    
    #pllotting rolling statistics
    orig=plt.plot(tseries,color='blue',label='Original')
    mean=plt.plot(rolmean,color='red',label='Rolling mean')
    std=plt.plot(rolstd,color='green',label='Roling std')
    plt.legend(loc='best')
    plt.title("rolling mean and std")
    plt.show(block=False)
    
    from statsmodels.tsa.stattools import adfuller
    print('Result of ADf test')
    dftest=adfuller(tseries['Confirmed'],autolag='AIC')
    dfoutput=pd.Series(dftest[0:4],index=['test statistic','p-value','#lag used','Number of observations used'])
    for key,value in dftest[4].items():
        dfoutput['critical valur(%s)'%key]=value
    print(dfoutput)
    
    

In [ ]:
test_stationarity(ind2)

# Diffrencing simple moving average for stationaeizing

In [ ]:
movingAvg=ind2.rolling(window=12).mean()
indMA=ind2-movingAvg
indMA.head(12)

In [ ]:
movingAvg=movingAvg.dropna()
movingAvg

In [ ]:
indMA.dropna(inplace=True)
indMA.head()

In [ ]:
test_stationarity(indMA)# testing stationarity and now the data ia stationary

# ARIMA MODEL

In [ ]:
#USING indMEA

from statsmodels.tsa.stattools import arma_order_select_ic
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
arma_order_select_ic(indMA)# checking the laags to use in ARIMA model

In [ ]:
model=ARIMA(indMA,order=(4,0,2))
results_arima=model.fit(disp=1)
plt.plot(indMA)
plt.plot(results_arima.fittedvalues,color='red')
plt.title('RSS: %4f'%sum(results_arima.fittedvalues-indMA['Confirmed']))
plt.show()

In [ ]:
model_fit=model.fit()

In [ ]:
predictMA=pd.Series(results_arima.fittedvalues,copy=True)
predictMA

In [ ]:
predMA=predictMA.to_frame()
predMA.head()

In [ ]:
#converting to original form by adding moving averages
modelvalues=predMA[0]+movingAvg['Confirmed']
modelvalues.tail()

In [ ]:
plt.plot(ind2)
plt.plot(modelvalues,color='red')
plt.show()

In [ ]:
ss=results_arima.forecast(20)
ss


In [ ]:
forcast,std,ci=ss

In [ ]:
type(results_arima)

In [ ]:
forcast

In [ ]:
std

In [ ]:
ci

In [ ]:
ind2.tail(12)

In [ ]:
sumof11=ind2.iloc[-11:].sum()

In [ ]:
sumof11

In [ ]:
ind3=ind2.tail(12)
ind3


In [ ]:
for i in forcast:
    s=ind3.iloc[-6:].sum()
    val=((7*i)+s)/6
    ind3=ind3.append(val,ignore_index=True)
ind3

In [ ]:
date=pd.date_range(start='05/2/2021', end='06/02/2021')

In [ ]:
date

In [ ]:
final= pd.DataFrame()
final["date"]=date

In [ ]:
final['Predicted']=ind3['Confirmed']

In [ ]:
flast

In [ ]:
flast['date']=pd.to_datetime(flast['date'],infer_datetime_format=True)
fn=flast.set_index(['date'])
fn

In [ ]:
orig=plt.plot(dfn,color='blue',label='Original')
predicted=plt.plot(fn,color='red',label='Predicted')

plt.legend(loc='best')
plt.title("original vs prediction")
plt.show(block=True)